Phone talking detection neural network.
Naive approach.
Pipeline:
1) Detect face on image
2) Crop Region of Interest (ROI) - face + const (80x80 pix)
3) Classify whether person talking on the phone or not

In [2]:
import cv2

import matplotlib.pyplot as plt
from tqdm import tqdm
from keras.models import load_model
import os
import numpy as np
import keras.backend as K

from sklearn.utils import shuffle
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D,ZeroPadding2D
from keras import regularizers
from keras.utils import np_utils
from keras.optimizers import SGD, Adadelta, Adagrad,Adam
from keras.callbacks import ModelCheckpoint
from six.moves import cPickle as pickle
from keras.utils.np_utils import to_categorical
import keras.backend as K
from six.moves import cPickle as pickle


from sklearn.utils import shuffle
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D,ZeroPadding2D
from keras import regularizers
from keras.utils import np_utils
from keras.optimizers import SGD, Adadelta, Adagrad,Adam
from keras.callbacks import ModelCheckpoint

import numpy as np
from keras.utils.np_utils import to_categorical

from mtcnn.mtcnn import MTCNN
import itertools
import keras
from keras.datasets import mnist

from keras.layers import Input, Dense, Add, Activation
from keras.models import Model

        
from keras.layers import Input, Dense, Add, Activation, BatchNormalization
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D,Conv1D, MaxPooling2D

from keras.layers import Input, Dense, Add, Activation
from keras.models import Model



Using TensorFlow backend.


The best results were achieved using CNN with one Inception block
https://medium.com/@sh.tsang/review-inception-v3-1st-runner-up-image-classification-in-ilsvrc-2015-17915421f77c

In [5]:
Y = np.load('/home/aevdakimov/Загрузки/Telegram Desktop/y.pickle', allow_pickle=True)
X = np.load('/home/aevdakimov/Загрузки/Telegram Desktop/X.pickle', allow_pickle=True)


X_open = np.load('/home/aevdakimov/Загрузки/Telegram Desktop/X_open.pickle', allow_pickle = True)
Y_open = np.load('/home/aevdakimov/Загрузки/Telegram Desktop/y_open.pickle', allow_pickle = True)


Y_train = np.hstack((np.array(Y_open),(np.array(Y))))
X_train = np.vstack((np.array(X_open),(np.array(X))))


Y_train = keras.utils.to_categorical(Y_train)


num_classes = 2
#X_train = X_train/255        


img_shape = 80
num_channels = 1

input_img = Input(shape=(img_shape, img_shape, num_channels))


X = input_img

X = Conv2D(64, (3,3), padding='same', activation='relu')(X)
X = MaxPooling2D(pool_size=(2, 2))(X)

way_0 = MaxPooling2D(pool_size=(2, 2))(X)

way_1 = Conv2D(64, (1,1), padding='same', activation='relu')(X)
way_1 = MaxPooling2D(pool_size=(2, 2))(way_1)

way_2 = Conv2D(64, (1,1), padding='same', activation='relu')(X)
way_3 = Conv2D(64, (3,3), padding='same', activation='relu')(way_2)
way_3 = MaxPooling2D(pool_size=(2, 2))(way_3)


way_4 = Conv2D(64, (1,1), padding='same', activation='relu')(X)
way_5 = Conv2D(64, (5,5), padding='same', activation='relu')(way_4)
way_5 = MaxPooling2D(pool_size=(2, 2))(way_5)


output = keras.layers.concatenate([way_0, way_1, way_3, way_5], axis = 3)


X = Flatten()(output)


X = Dense(128, activation='relu')(X)
X = Dropout(0.4)(X)

X = Dense(128, activation='relu')(X)
X = Dropout(0.4)(X)


predictions = Dense(1, activation='sigmoid')(X)



model = Model(inputs=input_img, outputs=predictions)
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model = Model(inputs=input_img, outputs=predictions)
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])
#model.fit(X_train, Y_train, batch_size=128,epochs=1)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 80, 80, 1)    0                                            
__________________________________________________________________________________________________
conv2d_7 (Conv2D)               (None, 80, 80, 64)   640         input_2[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_6 (MaxPooling2D)  (None, 40, 40, 64)   0           conv2d_7[0][0]                   
__________________________________________________________________________________________________
conv2d_9 (Conv2D)               (None, 40, 40, 64)   4160        max_pooling2d_6[0][0]            
__________________________________________________________________________________________________
conv2d_11 

In [3]:
# pretrained model

 model = load_model('/Users/user/Downloads/model_inception_v4_aug_64_64_64_64_64_64.model')

In [11]:
# Function detects face on the image, passes through NN,
# picture rectangle around ROI if predict is higher than threshold 
detector = MTCNN()

def phone(image, model, detector):

    crop_path = '/Users/user/Data_science/skytrack/imgs'

    pred = 0

    result = detector.detect_faces(image) # detecting face
    const = 50
    try:
        bounding_box = result[0]['box'] 
    
        x1 = bounding_box[0] - const
        x2 = (bounding_box[0] + bounding_box[2]) + const
        y1 = bounding_box[1]-const
        y2 = bounding_box[1]+bounding_box[3]+const
        
        crop_img = image[y1:y2, x1:x2] # cropping img
        
        tmp = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)
        tmp = tmp/255.
        tmp = cv2.resize(tmp, (80, 80)) # getting required ROI
        
        pred = model.predict(np.reshape(np.array(tmp),(1,80,80,1)))
        if pred >= 0.95: # threshold
            image = cv2.rectangle(image, (x1,y1), (x2,y2), (255,0,0), 3)
        print(pred)
    except Exception as e:
            print(e)
    
    return image

In [13]:
# Launcher 
import numpy as np 
import cv2   


cap = cv2.VideoCapture(0)

cap.set(3,640)
cap.set(4,480)
  
fourcc = cv2.VideoWriter_fourcc(*'MJPG') 
out = cv2.VideoWriter('output1.avi', fourcc, 30.0, (640, 480)) 

identifiers = {}
 
while(True): 
    ret, frame = cap.read()      
    image = frame    
    image = phone(image, model, detector)
    
    out.write(image)       
    cv2.imshow('phone', image) 
     
    if cv2.waitKey(1) & 0xFF == ord('a'): 
        break
          
cap.release() 
  
out.release()  
   
cv2.destroyAllWindows() 

[[0.5853238]]
[[0.3947083]]
[[0.13922094]]
[[0.13631833]]
[[0.1115536]]
[[0.12363242]]
[[0.13036759]]
[[0.1153552]]
[[0.29043055]]
[[0.11144801]]
[[0.17165452]]
[[0.1997334]]
[[0.12335743]]
[[0.13799615]]
[[0.1989433]]
[[0.72685194]]
[[0.35198867]]
[[0.08310848]]
[[0.85032237]]
[[0.68617177]]
[[0.10072633]]
[[0.4864713]]
[[0.3156617]]
[[0.10090802]]
[[0.1129461]]
[[0.3445109]]
[[0.8752569]]
[[0.4927609]]
[[0.3490645]]
[[0.33495468]]
[[0.06610416]]
[[0.09733833]]
[[0.10565314]]
[[0.10392562]]
[[0.62708074]]
[[0.8360292]]
[[0.9630065]]
[[0.9944436]]
[[0.99733716]]
[[0.9982151]]
[[0.99950683]]
[[0.99940073]]
[[0.9996164]]
[[0.99948007]]
[[0.99958724]]
[[0.99936885]]
[[0.9996439]]
[[0.9988992]]
[[0.99899405]]
[[0.9991443]]
[[0.96220577]]
[[0.30073088]]
[[0.09198131]]
[[0.20460051]]
[[0.30817315]]
[[0.31082642]]
[[0.31985247]]
[[0.43161705]]
[[0.27241582]]
[[0.24023423]]
[[0.8135438]]
[[0.88844085]]
[[0.7066775]]
[[0.96450174]]
[[0.747187]]
[[0.9866253]]
[[0.9913747]]
[[0.99527127]]
[[0.943